# IT Ticket Data Analysis

Interactive analysis and visualization of IT support ticket data using Python in JupyterLite.

This notebook demonstrates:
- Data loading and preprocessing
- Interactive visualizations with Plotly
- Statistical analysis with Altair
- Dashboard-style widgets for exploration

## Install Required Packages

In [ ]:
%pip install -q plotly altair ipywidgets pandas numpy nbformat pyarrow

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import altair as alt
from ipywidgets import interact, Dropdown, IntSlider
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

## Load and Explore the Data

In [ ]:
# Load the ticket data
df = pd.read_parquet('../data/exports/freshservice/all-tickets.parquet')

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {len(df.columns)}")
df.head()

In [ ]:
# Unnest, clean, and transform the data to match the expected notebook schema

# 1. Normalize the 'data' column which contains the full JSON payload
df_normalized = pd.json_normalize(df['data'])

# 2. Create the main DataFrame from the 'ticket' fields, removing the 'ticket.' prefix
df_ticket = df_normalized.filter(regex='^ticket\\.').rename(columns=lambda c: c.replace('ticket.', ''))
df_ticket.columns = df_ticket.columns.str.replace('^custom_fields\\.', '', regex=True)

# 3. Define a function to extract the first agent response time from conversations
def get_first_response_date(conversations):
    if not isinstance(conversations, list):
        return pd.NaT
    first_response_time = pd.Timestamp.max
    for conv in conversations:
        if isinstance(conv, dict) and not conv.get('incoming'):
            response_time = pd.to_datetime(conv.get('created_at'), errors='coerce')
            if response_time < first_response_time:
                first_response_time = response_time
    return first_response_time if first_response_time != pd.Timestamp.max else pd.NaT

# 4. Apply the function to create 'first_response_date'
if 'conversations' in df_normalized.columns:
    df_ticket['first_response_date'] = df_normalized['conversations'].apply(get_first_response_date)
else:
    df_ticket['first_response_date'] = pd.NaT

# 5. Rename date columns to match the old format
rename_map = {'created_at': 'created_date', 'updated_at': 'last_updated_date'}
df_ticket = df_ticket.rename(columns=rename_map, errors='ignore')

# 6. Define mappings for priority and status (based on observed data and common conventions)
priority_map = {1: 'Low', 2: 'Medium', 3: 'High', 4: 'Urgent'}
status_map = {
    1: 'Open',
    2: 'More Information Needed',
    3: 'Ready for Work',
    4: 'In Progress',
    5: 'Approvals Needed',
    6: 'Pending',
    7: 'Purchased',
    8: 'Transferred',
    9: 'Blocked Internal',
    10: 'Blocked External',
    11: 'Review'
}

# 7. Create 'resolved_date' and 'closed_date' and map status/priority to strings
if 'status' in df_ticket.columns:
    df_ticket['status_str'] = df_ticket['status'].map(status_map)
    if 'last_updated_date' in df_ticket.columns:
        df_ticket['resolved_date'] = pd.to_datetime(df_ticket.apply(lambda row: row['last_updated_date'] if row['status_str'] == 'Resolved' else pd.NaT, axis=1), errors='coerce')
        df_ticket['closed_date'] = pd.to_datetime(df_ticket.apply(lambda row: row['last_updated_date'] if row['status_str'] == 'Closed' else pd.NaT, axis=1), errors='coerce')
    df_ticket['status'] = df_ticket['status_str'].fillna(df_ticket['status']) # Keep original number if no map found
    df_ticket = df_ticket.drop(columns=['status_str'])

if 'priority' in df_ticket.columns:
    df_ticket['priority'] = df_ticket['priority'].map(priority_map).fillna(df_ticket['priority'])

# 8. Overwrite the original dataframe
df = df_ticket

print("DataFrame shape after full processing:", df.shape)
df.head()

In [ ]:
# Data preprocessing
# Convert date columns to datetime
date_columns = ['last_updated_date', 'first_response_date', 'assigned_date',
                'created_date', 'resolved_date', 'closed_date']

for col in date_columns:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Basic statistics
print("Basic Dataset Information:")
print(f"Total tickets: {len(df)}")
print(f"Date range: {df['created_date'].min()} to {df['created_date'].max()}")
print(f"\nStatus distribution:")
print(df['status'].value_counts())
print(f"\nPriority distribution:")
print(df['priority'].value_counts())

## Interactive Dashboard: Ticket Status Overview

In [ ]:
# Create a comprehensive dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Ticket Status Distribution', 'Priority Levels',
                   'Tickets by Location', 'Category Breakdown'),
    specs=[[{"type": "pie"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "pie"}]]
)

# Status distribution (pie chart)
status_counts = df['status'].value_counts()
fig.add_trace(
    go.Pie(labels=status_counts.index, values=status_counts.values, name="Status"),
    row=1, col=1
)

# Priority distribution (bar chart)
priority_counts = df['priority'].value_counts()
colors = {'Critical': 'red', 'High': 'orange', 'Medium': 'yellow', 'Low': 'green'}
priority_colors = [colors.get(p, 'blue') for p in priority_counts.index]
fig.add_trace(
    go.Bar(x=priority_counts.index, y=priority_counts.values,
           marker_color=priority_colors, name="Priority"),
    row=1, col=2
)

# Location distribution (bar chart)
location_counts = df['location'].value_counts().head(8)
fig.add_trace(
    go.Bar(x=location_counts.values, y=location_counts.index,
           orientation='h', name="Location"),
    row=2, col=1
)

# Category distribution (pie chart)
category_counts = df['category'].value_counts()
fig.add_trace(
    go.Pie(labels=category_counts.index, values=category_counts.values, name="Category"),
    row=2, col=2
)

fig.update_layout(
    title_text="IT Ticket Dashboard Overview",
    height=800,
    showlegend=False
)

fig.show()

## Response Time Analysis

In [ ]:
# Calculate response times
df['response_time_hours'] = (df['first_response_date'] - df['created_date']).dt.total_seconds() / 3600
df['resolution_time_hours'] = (df['resolved_date'] - df['created_date']).dt.total_seconds() / 3600

# Response time by priority
fig = px.box(df.dropna(subset=['response_time_hours']),
             x='priority', y='response_time_hours',
             color='priority',
             title='Response Time Distribution by Priority',
             labels={'response_time_hours': 'Response Time (Hours)'})

fig.update_layout(height=500)
fig.show()

In [ ]:
# Resolution time analysis
resolution_data = df.dropna(subset=['resolution_time_hours'])

fig = px.scatter(resolution_data,
                x='response_time_hours', y='resolution_time_hours',
                color='priority', size='no._of_customers_impacted',
                hover_data=['subject', 'status', 'agent_name'],
                title='Response Time vs Resolution Time',
                labels={'response_time_hours': 'Response Time (Hours)',
                       'resolution_time_hours': 'Resolution Time (Hours)'})

fig.update_layout(height=600)
fig.show()

## Interactive Ticket Timeline

In [ ]:
# Create timeline visualization
timeline_data = []

for _, ticket in df.iterrows():
    # Created event
    timeline_data.append({
        'Ticket': f"#{ticket['id']}",
        'Start': ticket['created_date'],
        'Finish': ticket['first_response_date'] if pd.notna(ticket['first_response_date']) else ticket['created_date'] + timedelta(hours=1),
        'Resource': 'Response Time',
        'Priority': ticket['priority']
    })

    # Resolution event (if resolved)
    if pd.notna(ticket['resolved_date']):
        timeline_data.append({
            'Ticket': f"#{ticket['id']}",
            'Start': ticket['first_response_date'] if pd.notna(ticket['first_response_date']) else ticket['created_date'],
            'Finish': ticket['resolved_date'],
            'Resource': 'Resolution Time',
            'Priority': ticket['priority']
        })

timeline_df = pd.DataFrame(timeline_data)

# Create Gantt chart
fig = px.timeline(timeline_df, x_start="Start", x_end="Finish", y="Ticket",
                 color="Resource", hover_data=["Priority"],
                 title="Ticket Timeline: Response and Resolution")

fig.update_layout(height=600)
fig.show()

## Agent Performance Analysis

In [ ]:
# Agent workload and performance
agent_stats = df.groupby('agent_name').agg({
    'id': 'count',
    'response_time_hours': 'mean',
    'resolution_time_hours': 'mean',
    'survey_score': 'mean',
    'priority': lambda x: (x == 'Critical').sum() + (x == 'High').sum()
}).round(2)

agent_stats.columns = ['Total_Tickets', 'Avg_Response_Time', 'Avg_Resolution_Time', 'Avg_Survey_Score', 'High_Priority_Tickets']
agent_stats = agent_stats.reset_index()

# Agent performance bubble chart
fig = px.scatter(agent_stats,
                x='Avg_Response_Time', y='Avg_Resolution_Time',
                size='Total_Tickets', color='Avg_Survey_Score',
                hover_name='agent_name',
                hover_data=['High_Priority_Tickets'],
                title='Agent Performance: Response vs Resolution Time',
                labels={'Avg_Response_Time': 'Average Response Time (Hours)',
                       'Avg_Resolution_Time': 'Average Resolution Time (Hours)'},
                color_continuous_scale='RdYlGn')

fig.update_layout(height=600)
fig.show()

## Interactive Filtering with Widgets

In [ ]:
# Interactive filtering function
def analyze_tickets(status_filter='All', priority_filter='All', location_filter='All'):
    # Filter data based on selections
    filtered_df = df.copy()

    if status_filter != 'All':
        filtered_df = filtered_df[filtered_df['status'] == status_filter]

    if priority_filter != 'All':
        filtered_df = filtered_df[filtered_df['priority'] == priority_filter]

    if location_filter != 'All':
        filtered_df = filtered_df[filtered_df['location'] == location_filter]

    # Create summary statistics
    print(f"Filtered Results: {len(filtered_df)} tickets")
    print(f"Average Response Time: {filtered_df['response_time_hours'].mean():.2f} hours")
    print(f"Average Survey Score: {filtered_df['survey_score'].mean():.2f}")

    # Create visualization
    if len(filtered_df) > 0:
        fig = px.histogram(filtered_df, x='item_category',
                          title=f'Ticket Categories ({status_filter}, {priority_filter}, {location_filter})',
                          labels={'item_category': 'Category', 'count': 'Number of Tickets'})
        fig.update_layout(height=400)
        fig.show()
    else:
        print("No tickets match the selected filters.")

# Create interactive widgets
status_options = ['All'] + list(df['status'].unique())
priority_options = ['All'] + list(df['priority'].unique())
location_options = ['All'] + list(df['location'].unique())

interact(analyze_tickets,
         status_filter=Dropdown(options=status_options, value='All', description='Status:'),
         priority_filter=Dropdown(options=priority_options, value='All', description='Priority:'),
         location_filter=Dropdown(options=location_options, value='All', description='Location:'))

## Altair Statistical Visualizations

In [ ]:
# Enable Altair to render in JupyterLite
alt.data_transformers.enable('json')

# Create correlation heatmap using Altair
numeric_cols = ['response_time_hours', 'resolution_time_hours', 'survey_score',
                'public_notes_count', 'agent_reply_count', 'customer_reply_count']

# Prepare data for correlation matrix
corr_data = df[numeric_cols].corr().reset_index().melt('index')
corr_data.columns = ['var1', 'var2', 'correlation']

heatmap = alt.Chart(corr_data).mark_rect().encode(
    x=alt.X('var1:N', title=''),
    y=alt.Y('var2:N', title=''),
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='redblue', domain=[-1, 1])),
    tooltip=['var1', 'var2', 'correlation']
).properties(
    title='Correlation Matrix of Ticket Metrics',
    width=400,
    height=400
)

heatmap

In [ ]:
# Interactive scatter plot with brushing
brush = alt.selection(type='interval')

points = alt.Chart(df.dropna(subset=['response_time_hours', 'survey_score'])).mark_circle(size=60).encode(
    x=alt.X('response_time_hours:Q', title='Response Time (Hours)'),
    y=alt.Y('survey_score:Q', title='Survey Score'),
    color=alt.condition(brush, alt.Color('priority:N'), alt.value('lightgray')),
    tooltip=['id', 'subject', 'priority', 'status', 'agent_name']
).add_selection(
    brush
).properties(
    title='Response Time vs Survey Score (Brush to Filter)',
    width=500,
    height=400
)

# Bar chart showing priority distribution of selected points
bars = alt.Chart(df).mark_bar().encode(
    x=alt.X('count()', title='Count'),
    y=alt.Y('priority:N', title='Priority'),
    color='priority:N'
).transform_filter(
    brush
).properties(
    title='Priority Distribution of Selected Tickets',
    width=300,
    height=400
)

points | bars

## Summary Statistics Table

In [ ]:
# Create comprehensive summary table
summary_stats = []

for priority in df['priority'].unique():
    priority_data = df[df['priority'] == priority]

    stats = {
        'Priority': priority,
        'Total Tickets': len(priority_data),
        'Avg Response Time (hrs)': priority_data['response_time_hours'].mean(),
        'Avg Resolution Time (hrs)': priority_data['resolution_time_hours'].mean(),
        'Avg Survey Score': priority_data['survey_score'].mean(),
        'Resolved %': (priority_data['status'] == 'Resolved').mean() * 100,
        'Escalated %': (priority_data['resolution_escalated'] == 'Yes').mean() * 100
    }
    summary_stats.append(stats)

summary_df = pd.DataFrame(summary_stats).round(2)

# Create interactive table with Plotly
fig = go.Figure(data=[go.Table(
    header=dict(values=list(summary_df.columns),
                fill_color='paleturquoise',
                align='left',
                font_size=12),
    cells=dict(values=[summary_df[col] for col in summary_df.columns],
               fill_color='lavender',
               align='left',
               font_size=11))
])

fig.update_layout(
    title='Summary Statistics by Priority Level',
    height=300
)

fig.show()